In [32]:
import numpy as np
import pandas as pd
from sklearn.linear_model import LinearRegression
from sklearn.kernel_ridge import KernelRidge
from sklearn.linear_model import MultiTaskLassoCV
from sklearn.metrics import mean_squared_log_error
from statsmodels.tsa.deterministic import DeterministicProcess
from sklearn import preprocessing
from sklearn.tree import DecisionTreeRegressor
from sklearn.linear_model import HuberRegressor
from sklearn.linear_model import RidgeCV
from sklearn.model_selection import GridSearchCV
import lightgbm as lgb
import xgboost as xgb
from sklearn.model_selection import RandomizedSearchCV
from sklearn.preprocessing import LabelEncoder
from sklearn.linear_model import LassoCV
import catboost as cb
from catboost import CatBoostRegressor


In [33]:
# path to the dataset in Kaggle's notebookpath = '/content/gdrive/My Drive/Colab Notebooks/input/store-sales-time-series-forecasting/'
path = '../input/store-sales-time-series-forecasting/'

In [34]:
# read oil price
data_oil = pd.read_csv(path + 'oil.csv', parse_dates=['date'], infer_datetime_format=True, index_col='date')

########################################################################################################################
# TODO: compute data_oil['ma_oil'] as the moving average of data_oil['dcoilwtico'] with window size 7
# Hint: check the documentation of .rolling() method of pandas.DataFrame
########################################################################################################################
# data_oil['ma_oil'] = None # change 'None' to your answer
data_oil['ma_oil'] = data_oil.rolling(16).mean()
# data_oil['ma_oil'] = data_oil.interpolate()

# Create continguous moving average of oil prices
calendar = pd.DataFrame(index=pd.date_range('2013-01-01', '2017-08-31'))
########################################################################################################################
# TODO 1: merge two DataFrame instances (data_oil and calendar) such that the merged instances has the same indexes
# as calendar.
# TODO 2: replace each NaN in data_oil['ma_oil'] by the first non-null value before it.
# Hint: check the documentation of .merge() and .fillna() methods of pandas.DataFrame
########################################################################################################################
calendar = calendar.merge(data_oil, how='outer', left_index=True, right_index=True) # merging data_oil and calendar dataframes into calendar dataframe
calendar['ma_oil'].fillna(method='ffill', inplace=True) # replacing NaN values with the first non-NaN value before it - check what to do with the first few NaN values
calendar['ma_oil'].iloc[:9] = calendar['dcoilwtico'].head(9).mean() # replacing first few NaN values with the mean of the original values for those many days
del calendar['dcoilwtico']
calendar.head(30) # visualise calendar dataframe

,ma_oil
2013-01-01,93.12
2013-01-02,93.12
2013-01-03,93.12
2013-01-04,93.12
2013-01-05,93.12
2013-01-06,93.12
2013-01-07,93.12
2013-01-08,93.12
2013-01-09,93.12
2013-01-10,NaN


In [35]:
########################################################################################################################
# TODO: create a True/False feature calendar['wd'] to indicate whether each date is a workday (Monday-Friday) or not.
# Hint: check documentation of pandas.DatetimeIndex.dayofweek
########################################################################################################################
calendar['wd'] = calendar.index.dayofweek <= 4 # add days of week column to calendar dataframe
calendar.index.name = 'date'
calendar.index = calendar.index.to_period('D')

calendar.head(20) # visualize calendar
# len(calendar.index)

,ma_oil,wd
date,,
2013-01-01,93.12,True
2013-01-02,93.12,True
2013-01-03,93.12,True
2013-01-04,93.12,True
2013-01-05,93.12,False
2013-01-06,93.12,False
2013-01-07,93.12,True
2013-01-08,93.12,True
2013-01-09,93.12,True


In [36]:
df_train = pd.read_csv(path + 'train.csv',
                       usecols=['store_nbr', 'family', 'date', 'sales', 'onpromotion'],
                       dtype={'store_nbr': 'int', 'family': 'category', 'sales': 'float32', 'onpromotion': 'float32'},
                       parse_dates=['date'], infer_datetime_format=True)

df_train.date = df_train.date.dt.to_period('D')
df_train = df_train.set_index(['date']).sort_index()
df_train.reset_index(inplace=True)
df_train['family'] = df_train['family'].cat.codes
# df_train['store_nbr'] = df_train['store_nbr'].cat.codes
df_train.tail(5) # display some entries of the training data

,date,store_nbr,family,sales,onpromotion
3000883,2017-08-15,9,28,438.132996,0.0
3000884,2017-08-15,9,29,154.552994,1.0
3000885,2017-08-15,9,30,2419.729004,148.0
3000886,2017-08-15,9,31,121.000000,8.0
3000887,2017-08-15,9,32,16.000000,0.0


In [37]:
data_stores = pd.read_csv(path + 'stores.csv', infer_datetime_format=True, index_col='store_nbr')
data_stores['city'] = data_stores['city'].astype('category').cat.codes
data_stores['state'] = data_stores['state'].astype('category').cat.codes
data_stores['type'] = data_stores['type'].astype('category').cat.codes
data_stores.head(5)

,city,state,type,cluster
store_nbr,,,,
1,18,12,3,13
2,18,12,3,13
3,18,12,3,8
4,18,12,3,9
5,21,14,3,4


In [38]:
df_train = df_train.merge(data_stores, how='left',on='store_nbr')
df_train.head()

,date,store_nbr,family,sales,onpromotion,city,state,type,cluster
0,2013-01-01,1,0,0.0,0.0,18,12,3,13
1,2013-01-01,1,1,0.0,0.0,18,12,3,13
2,2013-01-01,1,2,0.0,0.0,18,12,3,13
3,2013-01-01,1,3,0.0,0.0,18,12,3,13
4,2013-01-01,1,4,0.0,0.0,18,12,3,13


In [39]:
data_holidays = pd.read_csv(path + 'holidays_events.csv', parse_dates=['date'], infer_datetime_format=True, index_col='date')
data_holidays.index = data_holidays.index.to_period('D')
data_holidays['type'] = data_holidays['type'].astype('category').cat.codes
data_holidays['locale'] = data_holidays['locale'].astype('category').cat.codes
data_holidays['locale_name'] = data_holidays['locale_name'].astype('category').cat.codes
data_holidays['description'] = data_holidays['description'].astype('category').cat.codes
data_holidays['transferred'] = data_holidays['transferred'].astype('category').cat.codes
data_holidays.head(5)

,type,locale,locale_name,description,transferred
date,,,,,
2012-03-02,3,0,15,25,0
2012-04-01,3,2,2,54,0
2012-04-12,3,0,3,18,0
2012-04-14,3,0,12,6,0
2012-04-21,3,0,19,8,0


In [40]:
df_train = pd.merge(df_train,data_holidays, how='left',on='date')
df_train.head()

,date,store_nbr,family,sales,onpromotion,city,state,type_x,cluster,type_y,locale,locale_name,description,transferred
0,2013-01-01,1,0,0.0,0.0,18,12,3,13,3.0,1.0,4.0,51.0,0.0
1,2013-01-01,1,1,0.0,0.0,18,12,3,13,3.0,1.0,4.0,51.0,0.0
2,2013-01-01,1,2,0.0,0.0,18,12,3,13,3.0,1.0,4.0,51.0,0.0
3,2013-01-01,1,3,0.0,0.0,18,12,3,13,3.0,1.0,4.0,51.0,0.0
4,2013-01-01,1,4,0.0,0.0,18,12,3,13,3.0,1.0,4.0,51.0,0.0


In [41]:
train = pd.merge(df_train,calendar, how='left',on='date')
y_train = train['sales']
train.drop(['sales'],axis=1,inplace=True)
train = train.drop('date', axis=1)

In [42]:
train.head(5)

,store_nbr,family,onpromotion,city,state,type_x,cluster,type_y,locale,locale_name,description,transferred,ma_oil,wd
0,1,0,0.0,18,12,3,13,3.0,1.0,4.0,51.0,0.0,93.12,True
1,1,1,0.0,18,12,3,13,3.0,1.0,4.0,51.0,0.0,93.12,True
2,1,2,0.0,18,12,3,13,3.0,1.0,4.0,51.0,0.0,93.12,True
3,1,3,0.0,18,12,3,13,3.0,1.0,4.0,51.0,0.0,93.12,True
4,1,4,0.0,18,12,3,13,3.0,1.0,4.0,51.0,0.0,93.12,True


In [43]:
y_train.tail(5)

3054343     438.132996
3054344     154.552994
3054345    2419.729004
3054346     121.000000
3054347      16.000000
Name: sales, dtype: float32

In [44]:
train.isnull().sum()

store_nbr            0
family               0
onpromotion          0
city                 0
state                0
type_x               0
cluster              0
type_y         2551824
locale         2551824
locale_name    2551824
description    2551824
transferred    2551824
ma_oil           58806
wd                   0
dtype: int64

In [45]:
train.dtypes

store_nbr        int64
family            int8
onpromotion    float32
city              int8
state             int8
type_x            int8
cluster          int64
type_y         float64
locale         float64
locale_name    float64
description    float64
transferred    float64
ma_oil         float64
wd                bool
dtype: object

In [46]:
train['type_y'] = train['type_x'].fillna("Normal").astype('category').cat.codes
train['locale'] = train['locale'].fillna("Normal").astype('category').cat.codes
train['locale_name'] = train['locale_name'].fillna("Normal").astype('category').cat.codes
train['description'] = train['description'].fillna("Normal").astype('category').cat.codes
train['transferred'] = train['transferred'].fillna(False).astype('category').cat.codes
train.head()

,store_nbr,family,onpromotion,city,state,type_x,cluster,type_y,locale,locale_name,description,transferred,ma_oil,wd
0,1,0,0.0,18,12,3,13,3,1,4,50,0,93.12,True
1,1,1,0.0,18,12,3,13,3,1,4,50,0,93.12,True
2,1,2,0.0,18,12,3,13,3,1,4,50,0,93.12,True
3,1,3,0.0,18,12,3,13,3,1,4,50,0,93.12,True
4,1,4,0.0,18,12,3,13,3,1,4,50,0,93.12,True


In [47]:
train.isnull().sum()

store_nbr          0
family             0
onpromotion        0
city               0
state              0
type_x             0
cluster            0
type_y             0
locale             0
locale_name        0
description        0
transferred        0
ma_oil         58806
wd                 0
dtype: int64

In [48]:
# Create continguous moving average of oil prices
stest = '2017-08-16'
etest = '2017-08-31'
calendar_test = pd.DataFrame(index=pd.date_range(stest,etest))
########################################################################################################################
# TODO 1: merge two DataFrame instances (data_oil and calendar) such that the merged instances has the same indexes
# as calendar.
# TODO 2: replace each NaN in data_oil['ma_oil'] by the first non-null value before it.
# Hint: check the documentation of .merge() and .fillna() methods of pandas.DataFrame
########################################################################################################################
calendar_test = calendar_test.merge(data_oil, how='left', left_index=True, right_index=True) # merging data_oil and calendar dataframes into calendar dataframe
calendar_test['ma_oil'].fillna(method='ffill', inplace=True) # replacing NaN values with the first non-NaN value before it - check what to do with the first few NaN values

calendar_test['ma_oil']  = calendar.loc[stest:etest]['ma_oil'].values # add ma_oil attribute to the training dataframe
calendar_test['wd']   = calendar.loc[stest:etest]['wd'].values # add wd attribute to the training dataframe
del calendar_test['dcoilwtico']
calendar_test.tail(16) # visualise calendar dataframe

,ma_oil,wd
2017-08-16,48.893125,True
2017-08-17,48.798750,True
2017-08-18,48.770000,True
2017-08-19,48.770000,False
2017-08-20,48.770000,False
2017-08-21,48.624375,True
2017-08-22,48.464375,True
2017-08-23,48.418125,True
2017-08-24,48.270625,True
2017-08-25,48.184375,True


In [49]:
########################################################################################################################
# TODO: create a True/False feature calendar['wd'] to indicate whether each date is a workday (Monday-Friday) or not.
# Hint: check documentation of pandas.DatetimeIndex.dayofweek
########################################################################################################################
# calendar_test['wd'] = calendar_test.index.dayofweek <= 4 # add days of week column to calendar dataframe
calendar_test.index.name = 'date'
calendar_test.reset_index(inplace=True)
calendar_test.date = calendar_test.date.dt.to_period('D')
calendar_test.tail(16) # visualize calendar

,date,ma_oil,wd
0,2017-08-16,48.893125,True
1,2017-08-17,48.798750,True
2,2017-08-18,48.770000,True
3,2017-08-19,48.770000,False
4,2017-08-20,48.770000,False
5,2017-08-21,48.624375,True
6,2017-08-22,48.464375,True
7,2017-08-23,48.418125,True
8,2017-08-24,48.270625,True
9,2017-08-25,48.184375,True


In [50]:
df_test = pd.read_csv(path + 'test.csv',
                       usecols=['store_nbr', 'family', 'date', 'onpromotion'],
                       dtype={'store_nbr': 'int', 'family': 'category', 'sales': 'float32', 'onpromotion': 'float32'},
                       parse_dates=['date'], infer_datetime_format=True)

df_test.date = df_test.date.dt.to_period('D')
df_test = df_test.set_index(['date']).sort_index()
df_test.reset_index(inplace=True)
df_test['family'] = df_test['family'].cat.codes
# df_test['store_nbr'] = df_test['store_nbr'].cat.codes
df_test.tail(5) # display some entries of the training data

,date,store_nbr,family,onpromotion
28507,2017-08-31,9,28,1.0
28508,2017-08-31,9,29,0.0
28509,2017-08-31,9,30,1.0
28510,2017-08-31,9,31,9.0
28511,2017-08-31,9,32,0.0


In [51]:
df_test = df_test.merge(data_stores, how='left',on='store_nbr')
df_test.head()

,date,store_nbr,family,onpromotion,city,state,type,cluster
0,2017-08-16,1,0,0.0,18,12,3,13
1,2017-08-16,1,1,0.0,18,12,3,13
2,2017-08-16,1,2,2.0,18,12,3,13
3,2017-08-16,1,3,20.0,18,12,3,13
4,2017-08-16,1,4,0.0,18,12,3,13


In [52]:
df_test = pd.merge(df_test,data_holidays, how='left',on='date')
df_test.head()

,date,store_nbr,family,onpromotion,city,state,type_x,cluster,type_y,locale,locale_name,description,transferred
0,2017-08-16,1,0,0.0,18,12,3,13,NaN,NaN,NaN,NaN,NaN
1,2017-08-16,1,1,0.0,18,12,3,13,NaN,NaN,NaN,NaN,NaN
2,2017-08-16,1,2,2.0,18,12,3,13,NaN,NaN,NaN,NaN,NaN
3,2017-08-16,1,3,20.0,18,12,3,13,NaN,NaN,NaN,NaN,NaN
4,2017-08-16,1,4,0.0,18,12,3,13,NaN,NaN,NaN,NaN,NaN


In [53]:
test = pd.merge(df_test,calendar_test,how='outer',on='date')
test = test.drop('date', axis=1)

test.tail(5)

,store_nbr,family,onpromotion,city,state,type_x,cluster,type_y,locale,locale_name,description,transferred,ma_oil,wd
28507,9,28,1.0,18,12,1,6,NaN,NaN,NaN,NaN,NaN,47.464375,True
28508,9,29,0.0,18,12,1,6,NaN,NaN,NaN,NaN,NaN,47.464375,True
28509,9,30,1.0,18,12,1,6,NaN,NaN,NaN,NaN,NaN,47.464375,True
28510,9,31,9.0,18,12,1,6,NaN,NaN,NaN,NaN,NaN,47.464375,True
28511,9,32,0.0,18,12,1,6,NaN,NaN,NaN,NaN,NaN,47.464375,True


In [54]:
test.isnull().sum()

store_nbr          0
family             0
onpromotion        0
city               0
state              0
type_x             0
cluster            0
type_y         26730
locale         26730
locale_name    26730
description    26730
transferred    26730
ma_oil             0
wd                 0
dtype: int64

In [55]:
test['type_y'] = test['type_y'].fillna("Normal").astype('category').cat.codes
test['locale'] = test['locale'].fillna("Normal").astype('category').cat.codes
test['locale_name'] = test['locale_name'].fillna("Normal").astype('category').cat.codes
test['description'] = test['description'].fillna("Normal").astype('category').cat.codes
test['transferred'] = test['transferred'].fillna(False).astype('category').cat.codes
test.tail(5)

,store_nbr,family,onpromotion,city,state,type_x,cluster,type_y,locale,locale_name,description,transferred,ma_oil,wd
28507,9,28,1.0,18,12,1,6,1,1,1,1,0,47.464375,True
28508,9,29,0.0,18,12,1,6,1,1,1,1,0,47.464375,True
28509,9,30,1.0,18,12,1,6,1,1,1,1,0,47.464375,True
28510,9,31,9.0,18,12,1,6,1,1,1,1,0,47.464375,True
28511,9,32,0.0,18,12,1,6,1,1,1,1,0,47.464375,True


In [56]:
test.isnull().sum()

store_nbr      0
family         0
onpromotion    0
city           0
state          0
type_x         0
cluster        0
type_y         0
locale         0
locale_name    0
description    0
transferred    0
ma_oil         0
wd             0
dtype: int64

In [57]:
print(len(df_test),len(test))

28512 28512


In [62]:
model = lgb.LGBMRegressor(objective='regression_l1', max_depth=20)
model.fit(train,y_train)

LGBMRegressor(max_depth=20, objective='regression_l1')

In [63]:
sales_pred = model.predict(test)
sales_pred[sales_pred < 0] = 0. # Sales should be >= 0

In [64]:
# Create submission
df_sub = pd.read_csv(path + 'sample_submission.csv', index_col='id')
df_sub.sales = sales_pred
df_sub.to_csv('submission.csv', index=True)